## Preparación de datos para predicción numérica en SageMaker

*Nota: esta práctica debe ser ejecutada en un Jupyter Notebook para PySpark sobre el servicio de Athena en AWS.*

<div style="background-color: #f0f8ff; padding: 10px;">

Ya diste el primer paso en la preparación de los datos para hacer predicción numérica en SageMaker, con la creación de columnas dummy, esto se refleja en los esquemas de metadatos antes y después del one hot encoding respectivamente:
</div>

<div style="display: flex; justify-content: space-between;background-color: #f0f8ff; padding: 10px;"> 
  <div style="width: 48%;">
    <h4>Metadatos antes de one hot encoding</h4>
    <pre style="background-color: #f0f8ff">
root
 ├── age: integer (nullable = true)
 ├── sex: string (nullable = true)
 ├── bmi: double (nullable = true)
 ├── children: integer (nullable = true)
 ├── smoker: string (nullable = true)
 ├── region: string (nullable = true)
 └── charges: double (nullable = true)
    </pre>
  </div>
  
  <div style="width: 48%;">
    <h4>Metadatos después de one hot encoding</h4>
    <pre style="background-color: #f0f8ff">
root
 ├── age: integer (nullable = false)
 ├── bmi: double (nullable = false)
 ├── children: integer (nullable = false)
 ├── charges: double (nullable = false)
 ├── sex_female: byte (nullable = false)
 ├── sex_male: byte (nullable = false)
 ├── smoker_no: byte (nullable = false)
 ├── smoker_yes: byte (nullable = false)
 ├── region_northeast: byte (nullable = false)
 ├── region_northwest: byte (nullable = false)
 ├── region_southeast: byte (nullable = false)
 └── region_southwest: byte (nullable = false)
    </pre>
  </div>
</div>

<div style="background-color: #f0f8ff; padding: 10px;">

No obstante, también es necesario colocar la variable objetivo al principio del DataFrame. SageMaker por la forma en como procesa los datos, toma el primer campo como la variable objetivo. También hay que dividir los datos en conjuntos de entrenamiento y validación.
</div>

### Librerías

In [1]:
# Utilitarios para modificar el esquema de metadatos
from pyspark.sql.types import StructType, StructField

#Importamos los tipos de datos que definiremos para cada campo
from pyspark.sql.types import StringType, IntegerType, DoubleType

#Importamos la librerIa de pandas compatible con entornos de clUster de Big Data
import pyspark.pandas as pd

#Por defecto un dataframe Pandas muestra 1000 registros
#Vamos a indicarle que solo muestre 20 para que no se sature el notebook
pd.set_option("display.max_rows", 20)

#Libreria para manipular los servicios de AWS
import boto3

Calculation started (calculation_id=eec8a212-8305-2bfb-e789-84e3e5df675c) in (session=a6c8a210-c3a9-418d-a131-318d3464bdcc). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


### Configuración para lectura de archivos 

In [2]:
#Nombre del bucket desde donde se lee el archivo
#IMPORTANTE: REEMPLAZAR "XXX" POR TUS INICIALES
bucket = "datasets-ibd-mjc"

Calculation started (calculation_id=78c8a212-d221-a411-0a97-d606bec8147f) in (session=a6c8a210-c3a9-418d-a131-318d3464bdcc). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [3]:
#Ruta dentro del archivo dentro del bucket
rutaDeArchivo = "data/insurance"

Calculation started (calculation_id=26c8a212-fede-1c2a-7857-fd0d731b2c5e) in (session=a6c8a210-c3a9-418d-a131-318d3464bdcc). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [4]:
#Definicion del esquema de metadatos
schema = StructType(
    [
        StructField("age", DoubleType(), True),
        StructField("sex", StringType(), True),
        StructField("bmi", DoubleType(), True),
        StructField("children", DoubleType(), True),
        StructField("smoker", StringType(), True),
        StructField("region", StringType(), True),
        StructField("charges", DoubleType(), True)
    ]
)

Calculation started (calculation_id=80c8a213-b93d-383f-7bf7-a16aa4d64b69) in (session=a6c8a210-c3a9-418d-a131-318d3464bdcc). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


<div style="background-color: #f0f8ff; padding: 10px;">

Observa que en el esquema de metadatos, ha sido definido para que la edad `"age"` y la cantidad de hijos `"children"` sean de tipo decimal con `DoubleType()`. Desde el punto de vista de optimización a nivel de memoria RAM, SageMaker considera todo número como decimal, de modo que no tiene sentido definir los números como enteros, incluso aunque a nivel del negocio, no tenga sentido que estos números no sean decimales, de cualquier forma SageMaker los va a cargar como decimales.
</div>

### Rutas al dataset

<div style="background-color: #f0f8ff; padding: 10px;">

Define la ruta al bucket en donde se encuentra el archivo `insurance.csv` a partir de las variables definidas para lectura. Luego crea 3 directorios para almacenar los datos procesados, y los datos particionados en entrenamiento y validación respectivamente.
</div>

In [6]:
#Definimos la ruta desde donde se lee el archivo de datos que se preparara
#En Python 3, podemos anteponer "f" en una cadena para reemplazar valores
rutaArchivoRaw = f"s3://{bucket}/{rutaDeArchivo}/"

#Verificamos
print(rutaArchivoRaw)

Calculation started (calculation_id=04c8a215-601b-6066-05a9-31db2c6bd2be) in (session=a6c8a210-c3a9-418d-a131-318d3464bdcc). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
s3://datasets-ibd-mjc/data/insurance/



<div style="background-color: #f0f8ff; padding: 10px;">

Define la ruta para almacenara el 100% de los registros del dataset preparado.
</div>

In [7]:
#Definimos la ruta en donde se almacenara el 100% de los registros del dataset preparado
rutaArchivoDataset = f"s3://{bucket}/{rutaDeArchivo}_dataset/"

#Verificamos
print(rutaArchivoDataset)

Calculation started (calculation_id=22c8a216-041f-25aa-88e3-20850b6001d8) in (session=a6c8a210-c3a9-418d-a131-318d3464bdcc). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
s3://datasets-ibd-mjc/data/insurance_dataset/



<div style="background-color: #f0f8ff; padding: 10px;">

Define la ruta para almacenar los registros del dataset de entrenamiento.
</div>

In [8]:
#Definimos la ruta en donde se almacenara los registros del dataset de entrenamiento
rutaArchivoDatasetTrain = f"s3://{bucket}/{rutaDeArchivo}_train/"

#Verificamos
print(rutaArchivoDatasetTrain)

Calculation started (calculation_id=9cc8a216-15b7-9e96-c462-bcb014e8503e) in (session=a6c8a210-c3a9-418d-a131-318d3464bdcc). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
s3://datasets-ibd-mjc/data/insurance_train/



<div style="background-color: #f0f8ff; padding: 10px;">

Define la ruta para almacenar los registros del dataset de validación.
</div>

In [9]:
#Definimos la ruta en donde se almacenara los registros del dataset de validacion
rutaArchivoDatasetTest = f"s3://{bucket}/{rutaDeArchivo}_test/"

#Verificamos
print(rutaArchivoDatasetTest)

Calculation started (calculation_id=5ec8a216-258b-6aea-51e3-873ef4ae89cd) in (session=a6c8a210-c3a9-418d-a131-318d3464bdcc). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
s3://datasets-ibd-mjc/data/insurance_test/



<div style="background-color: #f0f8ff; padding: 10px;">

Lee los datos en crudo en la ruta definida por la variable `rutaArchivoRaw` con las configuraciones de encabezado, delimitador, encoding y esquema. Visualiza que es esquela de metadatos está correctamente configurado y visualiza los primeros 20 registros.
</div>

### Lectura de Datos

In [10]:
#Leemos los datos
dfRaw = spark.read.format("csv").option("header", "true").option("delimiter", ",").option("encoding", "ISO-8859-1").schema(schema).load(rutaArchivoRaw)

#Vemos el esquema de metadatos
dfRaw.printSchema()

#Verificamos
dfRaw.show()

Calculation started (calculation_id=aec8a216-82d9-4e5c-48f5-a759f9f0af8d) in (session=a6c8a210-c3a9-418d-a131-318d3464bdcc). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
root
 |-- age: double (nullable = true)
 |-- sex: string (nullable = true)
 |-- bmi: double (nullable = true)
 |-- children: double (nullable = true)
 |-- smoker: string (nullable = true)
 |-- region: string (nullable = true)
 |-- charges: double (nullable = true)

+----+------+------+--------+------+---------+-----------+
| age|   sex|   bmi|children|smoker|   region|    charges|
+----+------+------+--------+------+---------+-----------+
|19.0|female|  27.9|     0.0|   yes|southwest|  16884.924|
|18.0|  male| 33.77|     1.0|    no|southeast|  1725.5523|
|28.0|  male|  33.0|     3.0|    no|southeast|   4449.462|
|33.0|  male|22.705|     0.0|    no|northwest|21984.47061|
|32.0|  male| 28.88|     0.0|    no|northwest|  3866.8552|
|31.0|female| 25.74|     0.0|    no|southeast|  3756.6216|
|46.0|female| 33.44|     1.0|    no|southeast|  8240.5896|
|37.0|female| 27.74|     3.0|    no|northwest|  7281.5056|
|37.0|  male| 29.83|     2.0|    no|northeast|  6406.4107|
|60

### Conversion de Dataframe Spark a Pandas

<div style="background-color: #f0f8ff; padding: 10px;">

Convierte de DataFrame Spark a DataFrame Pandas y verifica que se ha transformado correctamente.
</div>

In [11]:
#Convertimos el dataframe SPARK a un dataframe PANDAS
dfpRaw = pd.from_pandas(dfRaw.toPandas())

#Verificamos
dfpRaw

Calculation started (calculation_id=98c8a216-c3ab-74f6-1cbe-b0a937fa0446) in (session=a6c8a210-c3a9-418d-a131-318d3464bdcc). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
     age     sex     bmi  children smoker     region      charges
0   19.0  female  27.900       0.0    yes  southwest  16884.92400
1   18.0    male  33.770       1.0     no  southeast   1725.55230
2   28.0    male  33.000       3.0     no  southeast   4449.46200
3   33.0    male  22.705       0.0     no  northwest  21984.47061
4   32.0    male  28.880       0.0     no  northwest   3866.85520
5   31.0  female  25.740       0.0     no  southeast   3756.62160
6   46.0  female  33.440       1.0     no  southeast   8240.58960
7   37.0  female  27.740       3.0     no  northwest   7281.50560
8   37.0    male  29.830       2.0     no  northeast   6406.41070
9   60.0  female  25.840       0.0     no  northwest  28923.13692
10  25.0    male  26.220       0.0     no  northeast   2721.32080
11  62.0  female  26.290       0.0    yes  southeast  27808.72510
12  23.0    male  34.400       0.0     no  southwest   1826.84300
13  56.0  female  39.820       0.0     no  southeast 

### Creacion de columnas Dummy

<div style="background-color: #f0f8ff; padding: 10px;">

Usa el utilitario `get_dummies` de Pandas para la creación de las columnas dummy. Verifica que se han creado de forma correcta.
</div>

In [12]:
#Conversion de variables categoricas a variables numericas en columnas dummy
dfpDataset = pd.get_dummies(dfpRaw, columns = [
  "sex",
  "region",
  "smoker"
])

#Verificamos
dfpDataset

Calculation started (calculation_id=54c8a216-e048-97bd-1bad-c820a4bb25a9) in (session=a6c8a210-c3a9-418d-a131-318d3464bdcc). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
     age     bmi  children      charges  sex_female  sex_male  region_northeast  region_northwest  region_southeast  region_southwest  smoker_no  smoker_yes
0   19.0  27.900       0.0  16884.92400           1         0                 0                 0                 0                 1          0           1
1   18.0  33.770       1.0   1725.55230           0         1                 0                 0                 1                 0          1           0
2   28.0  33.000       3.0   4449.46200           0         1                 0                 0                 1                 0          1           0
3   33.0  22.705       0.0  21984.47061           0         1                 0                 1                 0                 0          1           0
4   32.0  28.880       0.0   3866.85520           0         1                 0                 1                 0                 0          1           0
5   31.0  25.740       0.0   3756.6

### Conversion de Dataframe Pandas a Spark

<div style="background-color: #f0f8ff; padding: 10px;">

Convierte de nuevo el DataFrame Pandas a DataFrame Spark y visualiza que se haya hecho de forma correcta.
</div>

In [13]:
#Convertimos el dataframe PANDAS a un dataframe SPARK
dfDataset = dfpDataset.to_spark()

#Verificamos
dfDataset.show()

Calculation started (calculation_id=c8c8a217-0512-e54c-1c55-2b42c6ad3512) in (session=a6c8a210-c3a9-418d-a131-318d3464bdcc). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
+----+------+--------+-----------+----------+--------+----------------+----------------+----------------+----------------+---------+----------+
| age|   bmi|children|    charges|sex_female|sex_male|region_northeast|region_northwest|region_southeast|region_southwest|smoker_no|smoker_yes|
+----+------+--------+-----------+----------+--------+----------------+----------------+----------------+----------------+---------+----------+
|19.0|  27.9|     0.0|  16884.924|         1|       0|               0|               0|               0|               1|        0|         1|
|18.0| 33.77|     1.0|  1725.5523|         0|       1|               0|               0|               1|               0|        1|         0|
|28.0|  33.0|     3.0|   4449.462|         0|       1|               0|               0|               1|               0|        1|         0|
|33.0|22.705|     0.0|21984.47061|         0|       1|               0|               1|               0|        

### Orden de Columnas

<div style="background-color: #f0f8ff; padding: 10px;">

Observa que la primera columna es la edad `"age"`. Es necesario reordenar las columnas colocando la columna correspondiente a la variable objetivo `"charges"` en la primera posición. Esto dado que SageMaker requiere la variable objetivo en la primera posición. Esto se puede hacer con la función `select()` pasando como argumento el orden requerido para las columnas.
</div>

In [14]:
#Colocamos hasta el final la columna LABEL
dfDatasetOrdenado = dfDataset.select(
    dfDataset["charges"],
    dfDataset["age"],
    dfDataset["bmi"],
    dfDataset["children"],
    dfDataset["sex_female"],
    dfDataset["sex_male"],
    dfDataset["region_northeast"],
    dfDataset["region_northwest"],
    dfDataset["region_southeast"],
    dfDataset["region_southwest"],
    dfDataset["smoker_no"],
    dfDataset["smoker_yes"]
)

Calculation started (calculation_id=f6c8a217-99bb-53fa-1735-c3c15b4f4e33) in (session=a6c8a210-c3a9-418d-a131-318d3464bdcc). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


<div style="background-color: #f0f8ff; padding: 10px;">

Verifica que las columnas se han reordenado de forma correcta.
</div>

In [15]:
#Verificamos
dfDatasetOrdenado.show()

Calculation started (calculation_id=76c8a217-bc55-698c-996c-0cbd2c60a9f4) in (session=a6c8a210-c3a9-418d-a131-318d3464bdcc). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
+-----------+----+------+--------+----------+--------+----------------+----------------+----------------+----------------+---------+----------+
|    charges| age|   bmi|children|sex_female|sex_male|region_northeast|region_northwest|region_southeast|region_southwest|smoker_no|smoker_yes|
+-----------+----+------+--------+----------+--------+----------------+----------------+----------------+----------------+---------+----------+
|  16884.924|19.0|  27.9|     0.0|         1|       0|               0|               0|               0|               1|        0|         1|
|  1725.5523|18.0| 33.77|     1.0|         0|       1|               0|               0|               1|               0|        1|         0|
|   4449.462|28.0|  33.0|     3.0|         0|       1|               0|               0|               1|               0|        1|         0|
|21984.47061|33.0|22.705|     0.0|         0|       1|               0|               1|               0|        

### Division de datos para entrenar y validar

<div style="background-color: #f0f8ff; padding: 10px;">

Para hacer la división en conjunto de entrenamiento y validadción, usas la función `randomSplit()` en Spark el cual recibe una lista con los porcentajes de datos para entrenamiento y para validación. Este corte se hace de manera aleatoria. Esta función devuelve dos variables que por estándar se llaman `dfTrain` y `dfTest`.
</div>

In [16]:
#Division en datos para entrenar y validar
dfTrain, dfTest = dfDatasetOrdenado.randomSplit([0.8, 0.2])

Calculation started (calculation_id=eac8a218-0891-81c0-afd4-8542fedf1006) in (session=a6c8a210-c3a9-418d-a131-318d3464bdcc). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


<div style="background-color: #f0f8ff; padding: 10px;">

Verifica la cantidad de registros que hay en cada DataFrame `dfTrain` y `dfTest`. Un DataFrame de Spark posee la función `count()` la cual te permite contar la cantidad de registros que posee el DataFrame.
</div>

In [19]:
#Verificamos la cantidad de registros para entrenar
cantidadDeRegistrosTrain = dfTrain.count()

#Verificamos
print(cantidadDeRegistrosTrain)

Calculation started (calculation_id=f2c8a219-0f3c-e02a-db80-1838de9ee7ee) in (session=a6c8a210-c3a9-418d-a131-318d3464bdcc). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
1079



In [20]:
#Verificamos la cantidad de registros para validar
cantidadDeRegistrosTest = dfTest.count()

#Verificamos
print(cantidadDeRegistrosTest)

Calculation started (calculation_id=74c8a219-2157-28ff-53a6-80edbf3ebdde) in (session=a6c8a210-c3a9-418d-a131-318d3464bdcc). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
259



### Almacenamiento de dataframes

<div style="background-color: #f0f8ff; padding: 10px;">

Es recomendable que además de guardar los DataFrames de entrenamiento y de test, guardes el DataFrame completo. La partición en conjuntos de entrenamiento y test es una manera introductoria de evaluar tu modelo final sin embargo, lo que se usa en las empresas es la validación cruzada e hiperparametrización que se realiza sobre el DataFrame completo.
</div>

In [21]:
#Almacenamos el dataframe que tiene el 100% de registros
dfDatasetOrdenado.write.format("csv").option("header", "false").option("delimiter", ",").option("encoding", "ISO-8859-1").mode("overwrite").save(rutaArchivoDataset)

Calculation started (calculation_id=a4c8a219-be92-4e42-4f43-9152da237107) in (session=a6c8a210-c3a9-418d-a131-318d3464bdcc). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [22]:
#Almacenamos el dataframe de entrenamiento
dfTrain.write.format("csv").option("header", "false").option("delimiter", ",").option("encoding", "ISO-8859-1").mode("overwrite").save(rutaArchivoDatasetTrain)

Calculation started (calculation_id=58c8a219-d8b4-6613-74e8-fd1cea4735e2) in (session=a6c8a210-c3a9-418d-a131-318d3464bdcc). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [23]:
#Almacenamos el dataframe de validacion
dfTest.write.format("csv").option("header", "false").option("delimiter", ",").option("encoding", "ISO-8859-1").mode("overwrite").save(rutaArchivoDatasetTest)

Calculation started (calculation_id=dec8a219-f056-597d-6b39-b689bb47cc42) in (session=a6c8a210-c3a9-418d-a131-318d3464bdcc). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


<div style="background-color: #f0f8ff; padding: 10px;">

Dirígete al servicio de S3 y verifica que los DataFrames se han guardado de manera exitosa en las carpetas destinadas para cada uno.
</div>

### Eliminacion de archivo "_SUCCESS"

<div style="background-color: #f0f8ff; padding: 10px;">

Cada directorio donde almacenaste los DataFrames poseen un archivo `_SUCCESS` que debe ser eliminado. Esto lo puedes hacer usando la librería `boto3`.
</div>

In [24]:
#Nos conectamos al servicio de "S3"
s3 = boto3.client("s3")

Calculation started (calculation_id=c8c8a21a-490f-85cb-d029-fc4606da919a) in (session=a6c8a210-c3a9-418d-a131-318d3464bdcc). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [25]:
#Eliminamos el archivo "_SUCCESS" del dataset en donde se encuentran el 100% de registros
s3.delete_object(
    Bucket = bucket,
    Key = f"{rutaDeArchivo}_dataset/_SUCCESS"
)

Calculation started (calculation_id=a6c8a21a-7ff5-0375-f23e-432ed8cd3f85) in (session=a6c8a210-c3a9-418d-a131-318d3464bdcc). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
{'ResponseMetadata': {'RequestId': '0CKR547ZDCQ1VT15', 'HostId': 'AVSSOfugZH0c1o+mn9U1Pc1bGv4xzdebT85+EmZnCFawIeQ7fIRQTvHJLVc56z9S6/Xsa5zh60M=', 'HTTPStatusCode': 204, 'HTTPHeaders': {'x-amz-id-2': 'AVSSOfugZH0c1o+mn9U1Pc1bGv4xzdebT85+EmZnCFawIeQ7fIRQTvHJLVc56z9S6/Xsa5zh60M=', 'x-amz-request-id': '0CKR547ZDCQ1VT15', 'date': 'Mon, 12 Aug 2024 01:30:10 GMT', 'server': 'AmazonS3'}, 'RetryAttempts': 0}}



In [26]:
#Eliminamos el archivo "_SUCCESS" del dataset de entrenamiento
s3.delete_object(
    Bucket = bucket,
    Key = f"{rutaDeArchivo}_train/_SUCCESS"
)

Calculation started (calculation_id=6cc8a21a-9424-393b-710e-122f34ca0f21) in (session=a6c8a210-c3a9-418d-a131-318d3464bdcc). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
{'ResponseMetadata': {'RequestId': '8E4802W256C48NDB', 'HostId': 'OFKlYL6OEyQHoAqCZYgPsPWmCrSzuxkgOyWwpHv1aYHjwjfKaw4HKzRGfornT8l37CJSk1BbEuU=', 'HTTPStatusCode': 204, 'HTTPHeaders': {'x-amz-id-2': 'OFKlYL6OEyQHoAqCZYgPsPWmCrSzuxkgOyWwpHv1aYHjwjfKaw4HKzRGfornT8l37CJSk1BbEuU=', 'x-amz-request-id': '8E4802W256C48NDB', 'date': 'Mon, 12 Aug 2024 01:30:21 GMT', 'server': 'AmazonS3'}, 'RetryAttempts': 0}}



In [27]:
#Eliminamos el archivo "_SUCCESS" del dataset de validacion
s3.delete_object(
    Bucket = bucket,
    Key = f"{rutaDeArchivo}_test/_SUCCESS"
)

Calculation started (calculation_id=1ec8a21a-b378-a871-8ab5-5cc51dbd1c9c) in (session=a6c8a210-c3a9-418d-a131-318d3464bdcc). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
{'ResponseMetadata': {'RequestId': 'VBKGGVNRZR1BBVYR', 'HostId': 'Hlx/6rHR8XlsK7/CKGJMxMHFHwiV3fUijfmxwNhuMZEySBcl09n/ZilFJobIjXfF2/0iwHWQSSGxMdtGNvedcdbIcybDv++bUXlAQZDnckw=', 'HTTPStatusCode': 204, 'HTTPHeaders': {'x-amz-id-2': 'Hlx/6rHR8XlsK7/CKGJMxMHFHwiV3fUijfmxwNhuMZEySBcl09n/ZilFJobIjXfF2/0iwHWQSSGxMdtGNvedcdbIcybDv++bUXlAQZDnckw=', 'x-amz-request-id': 'VBKGGVNRZR1BBVYR', 'date': 'Mon, 12 Aug 2024 01:30:37 GMT', 'server': 'AmazonS3'}, 'RetryAttempts': 0}}



<div style="background-color: #f0f8ff; padding: 10px;">

Con esta preparación de los datos ya puedes crear un modelo de regresión lineal simple en SageMaker o inclusive un modelo más avanzado de redes neuronales. Por otro lado, si estuvieras en un modelo de predicción categórica, la preparación de los datos es diferente. Depende de 4 cosas. Estás haciendo predicción numérica o categórica. El modelo que usas se basa en ecuaciones o en tablas estadísticas. Entonces tienes 4 posibles combinaciones para la preparación de los datos.  
</div>